In [13]:
import os

In [14]:
%pwd

'c:\\Users\\B HEMANTH\\OneDrive\\Desktop\\e2e_text_summary\\end2end_text_summarization\\research'

In [15]:
os.chdir("../")

In [16]:
%pwd

'c:\\Users\\B HEMANTH\\OneDrive\\Desktop\\e2e_text_summary\\end2end_text_summarization'

In [17]:
from pathlib import Path

PROJECT_ROOT = Path.cwd()

CONFIG_FILE_PATH = PROJECT_ROOT / "config" / "config.yaml"
PARAMS_FILE_PATH = PROJECT_ROOT / "params.yaml"

print(CONFIG_FILE_PATH)
print(CONFIG_FILE_PATH.exists())

c:\Users\B HEMANTH\OneDrive\Desktop\e2e_text_summary\end2end_text_summarization\config\config.yaml
True


In [18]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path

In [19]:
# from textsummarizer.constants import *
from textsummarizer.utils.common import read_yaml, create_directories


In [20]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_url=config.source_url,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [21]:
import os
import urllib.request as request
import zipfile
from textsummarizer.logging import logger
from textsummarizer.utils.common import get_size


In [22]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_url,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  

        
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [23]:
from pathlib import Path

path = Path("../config/config.yaml").exists()
print(path)

False


In [24]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2026-03-01 14:59:52,023: INFO: common: yaml file: c:\Users\B HEMANTH\OneDrive\Desktop\e2e_text_summary\end2end_text_summarization\config\config.yaml loaded successfuly]
[2026-03-01 14:59:52,026: INFO: common: yaml file: c:\Users\B HEMANTH\OneDrive\Desktop\e2e_text_summary\end2end_text_summarization\params.yaml loaded successfuly]
[2026-03-01 14:59:52,031: INFO: common: created directory at: artifacts]
[2026-03-01 14:59:52,032: INFO: common: created directory at: artifacts/data_validation]
[2026-03-01 14:59:52,033: INFO: 499299850: File already exists of size: ~8181 KB]
